## 1.1 Detecting and Fixing Errors in dirty_data.csv 

General approach
Look for potential dirty data in the following areas:
- integrity constraints
- data entry error
- wrong categorical data
- violation of referential integrity
- duplicated data
- go against value range
- wrong encoding
- wrong representations
- wrong names and numbers

Import necessary libraries

In [4]:
import pandas as pd

In [5]:
dirty_data_path = "data\Group109_dirty_data.csv"
branch_path = r"data\branches.csv"
edges_path = "data\edges.csv"
nodes_path = r"data\nodes.csv"

dirty_df = pd.read_csv(dirty_data_path)
branch_df = pd.read_csv(branch_path)
edges_df = pd.read_csv(edges_path)
nodes_df = pd.read_csv(nodes_path)

In [6]:
branch_df

,branch_code,branch_name,branch_lat,branch_lon
0,NS,Nickolson,-37.773803,144.983647
1,TP,Thompson,-37.861835,144.905716
2,BK,Bakers,-37.815834,145.046450


In [7]:
dirty_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   order_id                 500 non-null    object 
 1   date                     500 non-null    object 
 2   time                     500 non-null    object 
 3   order_type               500 non-null    object 
 4   branch_code              500 non-null    object 
 5   order_items              500 non-null    object 
 6   order_price              500 non-null    float64
 7   customer_lat             500 non-null    float64
 8   customer_lon             500 non-null    float64
 9   customerHasloyalty?      500 non-null    int64  
 10  distance_to_customer_KM  500 non-null    float64
 11  delivery_fee             500 non-null    float64
dtypes: float64(5), int64(1), object(6)
memory usage: 47.0+ KB


In [8]:
dirty_df.describe()

,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,480.712900,-30.753946,143.504403,0.102000,8.629274,13.877162
std,254.034843,25.337436,16.299630,0.302951,1.596279,2.378285
min,31.750000,-37.827188,-37.822231,0.000000,3.613000,5.646222
25%,300.625000,-37.818738,144.952786,0.000000,7.750500,12.660927
50%,434.250000,-37.811755,144.963914,0.000000,8.639500,13.849738
75%,633.250000,-37.804505,144.980037,0.000000,9.633500,15.229668
max,1361.500000,145.005221,145.015449,1.000000,13.735000,20.088572


In [9]:
dirty_df.head()

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
0,ORDX02948,2018-03-11,10:11:49,Breakfast,BK,"[('Cereal', 4), ('Eggs', 3), ('Coffee', 1), ('...",206.00,-37.805040,144.963243,0,7.615,14.403915
1,ORDC07988,2018-03-06,10:01:41,Breakfast,NS,"[('Coffee', 2), ('Cereal', 5), ('Eggs', 10), (...",437.00,-37.815989,144.983435,0,8.914,13.807773
2,ORDI00568,2018-07-05,14:05:04,Lunch,NS,"[('Fries', 6), ('Chicken', 5), ('Salad', 7), (...",507.40,-37.806281,144.941960,1,9.316,15.028384
3,ORDI06756,2018-04-23,11:43:05,Breakfast,NS,"[('Coffee', 3), ('Pancake', 6), ('Eggs', 3)]",234.00,-37.822259,144.946977,0,9.975,15.217270
4,ORDX01986,2018-04-29,11:53:14,Breakfast,BK,"[('Coffee', 6), ('Cereal', 1), ('Eggs', 2), ('...",134.25,-37.815950,144.986001,0,6.038,13.677500


As stated in the assignment brief, we will not be looking for any fixing values within the following columns as there are no errors in dirty data for them:
- `order_id`
- `time`
- the numeric quantity in `order_items`
- `delivery_fee`

- [x] check if dates, time is properly formatted
- [x] check if branch code is correct
- [ ] check if order price makes sense according to number of items ordered
- [ ] check if distance to customer is calculated properly in shortest distance
- [ ] check if customer_lat and customer_lon exist in nodes
- [x] check if order_type is correct according to time
- [ ] check if customer does / does not have loyalty according to delivery fee ()


Delivery fee is calculated using a different method for each branch.
The fee depends linearly (but in different ways for each branch) on:
a. weekend or weekday (1 or 0) - as a continuous variable
b. time of the day (morning 0, afternoon 1, evening 2) - as a continuous variable
c. distance between branch and customer


In [10]:
dirty_df['branch_code'].value_counts()

TP    169
NS    163
BK    144
ns     11
tp      7
bk      6
Name: branch_code, dtype: int64

From the output above, we can see that some of the branch codes have been inputted in the wrong representation. Instead of being inputted in all uppercase, some of the values are shown as lowercase. We will need to perform data transformation on this column to convert all values to uppercase letters

In [11]:
dirty_df['branch_code'] = dirty_df['branch_code'].apply(str.upper)

According to the assignment brief, all string date values in column `date` should be in the format YYYY-MM-DD. We can verify if this is the case by using `pd.to_datetime` function on the `date` column and see if all date values fit the format `%Y-%m-%d`

In [12]:
try:
    pd.to_datetime(dirty_df['date'], format='%Y-%m-%d', errors='raise')
except ValueError as e:
    print("Error occurred. Unable to convert the following date:")
    print(e)
else:
    print('No error')

Error occurred. Unable to convert the following date:
time data 06-10-2018 doesn't match format specified


As we can see, not all the date values are in the correct format. Therefore we will have to iterate through each date value and determine which one of the follow formats the date value can be in:
1. YYYY-MM-DD
2. DD-MM-YYYY
3. YYYY-DD-MM

In [13]:
for index, row in dirty_df.iterrows():
    new_row = pd.Series(row)

    # Check if date fits the format YYYY-MM-DD
    try:
        new_row = pd.to_datetime(row['date'], format='%Y-%m-%d')
    except:
        # If not, check if it fits the format YYYY-DD-MM
        try: 
            new_row = pd.to_datetime(row['date'], format='%Y-%d-%m')
        # Else, check if it fits the format DD-MM-YYYY
        except:
            new_row = pd.to_datetime(row['date'], format='%d-%m-%Y')
    dirty_df.at[index, 'date'] = new_row

dirty_df['date'] = pd.to_datetime(dirty_df['date'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')

Now we evaluate the `order_type` column. Since we know for certain that the time column does not have any errors in it, we can verify if the `order_type` is correct according to the time. The order should be the following according to the times:
- 08:00:00 - 12:00:00 = Breakfast
- 12:00:01 - 16:00:00 = Lunch
- 16:00:01 - 20:00:00 = Dinner

In [17]:
dirty_df['order_type'].value_counts()

Breakfast    170
Lunch        165
Dinner       165
Name: order_type, dtype: int64

To check if the correct order type has been inputted, we create the function `find_order-type`. It looks through each string time value in `time` column, converts this string into datetime format. Then we check the timestamp to see if it fits within the Breakfast, Lunch or Dinner time slots

In [18]:
def find_order_type(time):
    timestamp = pd.to_datetime(time, format='%H:%M:%S').time()
    if timestamp >= pd.to_datetime('08:00:00').time() and timestamp <= pd.to_datetime('12:00:00').time():
        return 'Breakfast'
    elif timestamp >= pd.to_datetime('12:00:01').time() and timestamp < pd.to_datetime('16:00:00').time():
        return 'Lunch'
    elif timestamp >= pd.to_datetime('16:00:01').time() and timestamp <= pd.to_datetime('20:00:00').time():
        return 'Dinner'
    else:
        return 'Error'

In [25]:
# output values where calculated order type does not match the order type in the dataset
dirty_df[dirty_df['time'].apply(find_order_type) != dirty_df['order_type']].head()

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
14,ORDC09610,2018-10-06,17:07:36,Lunch,NS,"[('Fish&Chips', 5), ('Shrimp', 2), ('Pasta', 4)]",393.0,-37.826028,144.984514,0,9.578,17.378357
19,ORDC06273,2018-09-05,16:06:45,Lunch,NS,"[('Pasta', 5), ('Shrimp', 10), ('Fish&Chips', ...",1361.5,-37.814936,144.927351,0,10.813,16.896152
20,ORDB10659,2018-04-20,11:32:57,Dinner,TP,"[('Eggs', 7), ('Coffee', 10), ('Cereal', 6), (...",597.5,-37.818900,144.952797,0,8.576,11.161592
37,ORDK01676,2018-03-16,10:21:58,Dinner,BK,"[('Eggs', 2), ('Pancake', 2)]",92.5,-37.801158,144.957692,0,8.326,13.278789
54,ORDB10190,2018-10-08,17:17:44,Breakfast,TP,"[('Fish&Chips', 2), ('Salmon', 2)]",152.0,-37.813657,144.957285,0,8.419,13.088105


As shown above, there are orders in which order type has been incorrectly inputted according to the time. We can fix this issue by performing the following

In [26]:
dirty_df['order_type'] = dirty_df['time'].apply(find_order_type)

In [29]:
dirty_df['order_type'].value_counts()

Dinner       170
Breakfast    166
Lunch        164
Name: order_type, dtype: int64

Check if `distance_to_customer_KM` is correct

In [100]:
dirty_df.head()

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
0,ORDX02948,2018-03-11,10:11:49,Breakfast,BK,"[(Cereal, 4), (Eggs, 3), (Coffee, 1), (Pancake...",206.00,-37.805040,144.963243,0,7.615,14.403915
1,ORDC07988,2018-03-06,10:01:41,Breakfast,NS,"[(Coffee, 2), (Cereal, 5), (Eggs, 10), (Pancak...",437.00,-37.815989,144.983435,0,8.914,13.807773
2,ORDI00568,2018-07-05,14:05:04,Lunch,NS,"[(Fries, 6), (Chicken, 5), (Salad, 7), (Burger...",507.40,-37.806281,144.941960,1,9.316,15.028384
3,ORDI06756,2018-04-23,11:43:05,Breakfast,NS,"[(Coffee, 3), (Pancake, 6), (Eggs, 3)]",234.00,-37.822259,144.946977,0,9.975,15.217270
4,ORDX01986,2018-04-29,11:53:14,Breakfast,BK,"[(Coffee, 6), (Cereal, 1), (Eggs, 2), (Pancake...",134.25,-37.815950,144.986001,0,6.038,13.677500


In [98]:
import networkx as nx

In [103]:
G = nx.Graph()

In [104]:
G.add_nodes_from(nodes_df['node'])

In [113]:
branch_df

,branch_code,branch_name,branch_lat,branch_lon
0,NS,Nickolson,-37.773803,144.983647
1,TP,Thompson,-37.861835,144.905716
2,BK,Bakers,-37.815834,145.046450


In [109]:
for index, row in edges_df.iterrows():
    G.add_edge(row['u'], row['v'], weight=row['distance(m)'])

In [125]:
branch_lat = branch_df[branch_df['branch_code'] == dirty_df.loc[0]['branch_code']]['branch_lat'].values[0]
branch_lon = branch_df[branch_df['branch_code'] == dirty_df.loc[0]['branch_code']]['branch_lon'].values[0]


In [136]:
nodes_df['lon'] == branch_lon

0        False
1        False
2        False
3        False
4        False
         ...  
17112    False
17113    False
17114    False
17115    False
17116    False
Name: lon, Length: 17117, dtype: bool

In [139]:
nodes_df[(nodes_df['lat'] == branch_lat) & (nodes_df['lon'] == branch_lon)]

,node,lat,lon
10781,1889485053,-37.815834,145.04645


In [141]:
nodes_df[(nodes_df['lat'] == dirty_df.loc[0]['customer_lat']) & (nodes_df['lon'] == dirty_df.loc[0]['customer_lon'])]

,node,lat,lon
3347,1492410210,-37.80504,144.963243


In [142]:
shortest_path_length = nx.shortest_path_length(G, source=1889485053, target=1492410210, weight='weight')

In [145]:
shortest_path_length/1000

7.615

## Detecting and Removing Outlier Rows in outlier_data.csv

In [2]:
outlier_data_path = "data\Group109_outlier_data.csv"
outlier_df = pd.read_csv(outlier_data_path)
